In [3]:

import sys
sys.path.append('src/')
from src.logic_utils import get_lang

import torch
import numpy as np

In [4]:
lark_path = 'src/lark/exp.lark'
lang_base_path = 'data/lang/'
lang, clauses, bk, atoms = get_lang(
        lark_path, lang_base_path,'coinjump', 'coinjump')

In [5]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(obj1),
 jump(obj2),
 jump(obj3),
 jump(obj4),
 left_go_get_key(obj1),
 left_go_get_key(obj2),
 left_go_get_key(obj3),
 left_go_get_key(obj4),
 left_go_to_door(obj1),
 left_go_to_door(obj2),
 left_go_to_door(obj3),
 left_go_to_door(obj4),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj

In [1]:

from percept import SlotAttentionPerceptionModule, YOLOPerceptionModule
from facts_converter import FactsConverter
from nsfr import NSFReasoner
from logic_utils import build_infer_module, generate_atoms
from valuation import RLValuationModule
device = torch.device('cpu')

ModuleNotFoundError: No module named 'percept'

In [ ]:
def get_nsfr_model(lang, clauses, atoms, bk, device):
    PM = YOLOPerceptionModule(e=4, d=11, device=device)
    VM = RLValuationModule(
            lang=lang, device=device)
    FC = FactsConverter(lang=lang, perception_module=PM,
                        valuation_module=VM, device=device)
    IM = build_infer_module(clauses, atoms, lang,
                            m=len(clauses), infer_step=2, device=device)
    # Neuro-Symbolic Forward Reasoner
    NSFR = NSFReasoner(perception_module=PM, facts_converter=FC,
                       infer_module=IM, atoms=atoms, bk=bk, clauses=clauses)
    return NSFR

In [ ]:
NSFR = get_nsfr_model(lang, clauses, atoms, bk, device=device)

In [2]:
metric =[[0,0,0,1,0.21,0.2],[1,0,0,0, 0.18, 0.22],[0,0,1,0,0.9,0.8],[0,1,0,0,0.5,0.5]]
x = torch.tensor(np.array(metric), dtype=torch.float32).unsqueeze(0)

NameError: name 'torch' is not defined

In [8]:
x.shape

torch.Size([1, 4, 6])

In [9]:
V_T = NSFR(x)

In [10]:
NSFR.print_valuation_batch(V_T)

====== LEARNED PROGRAM ======
C_0:  jump(O1):-type(O1,agent),type(O2,enemy),closeby(O1,O2). 1.0
===== BATCH:  0 =====
15 jump(obj2) :  1.0
33 type(obj4,key) :  0.993
1 .(__T__) :  0.993
27 type(obj3,door) :  0.993
22 type(obj2,agent) :  0.993
20 type(obj1,enemy) :  0.993
2 closeby(obj1,obj2) :  0.993
5 closeby(obj2,obj1) :  0.993


In [11]:
V_T

tensor([[0.02183, 0.99322, 0.99312, 0.02183, 0.02183, 0.99312, 0.02183, 0.02183, 0.02183, 0.02183, 0.02183, 0.02183, 0.02183, 0.02183, 0.02188, 1.00000, 0.02185, 0.02185, 0.02183, 0.02183, 0.99322, 0.02183, 0.99322, 0.02183, 0.02183, 0.02183, 0.02183, 0.99322, 0.02183, 0.02183, 0.02183, 0.02183, 0.02183, 0.99322]],
       grad_fn=<DivBackward0>)

In [12]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 jump(obj1),
 jump(obj2),
 jump(obj3),
 jump(obj4),
 type(obj1,agent),
 type(obj1,door),
 type(obj1,enemy),
 type(obj1,key),
 type(obj2,agent),
 type(obj2,door),
 type(obj2,enemy),
 type(obj2,key),
 type(obj3,agent),
 type(obj3,door),
 type(obj3,enemy),
 type(obj3,key),
 type(obj4,agent),
 type(obj4,door),
 type(obj4,enemy),
 type(obj4,key)]

In [13]:
for i, atom in enumerate(atoms):
    print(V_T[0][i], atom)

tensor(0.02183, grad_fn=<SelectBackward0>) .(__F__)
tensor(0.99322, grad_fn=<SelectBackward0>) .(__T__)
tensor(0.99312, grad_fn=<SelectBackward0>) closeby(obj1,obj2)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj1,obj3)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj1,obj4)
tensor(0.99312, grad_fn=<SelectBackward0>) closeby(obj2,obj1)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj2,obj3)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj2,obj4)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj3,obj1)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj3,obj2)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj3,obj4)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj4,obj1)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj4,obj2)
tensor(0.02183, grad_fn=<SelectBackward0>) closeby(obj4,obj3)
tensor(0.02188, grad_fn=<SelectBackward0>) jump(obj1)
tensor(1., grad_fn=<SelectBackward0>) jump(obj2)
tensor(0.02185, grad_fn=<SelectBackward0>) jump(o